Run the cell below using 'shift + enter' which highlights the next cell. Be sure to execute the cells with text in them so the formatting is applied and they are able to be read.

In [20]:
from vpython import *
from Charge import Charge, euler
from Charge import runge_kutta_2 as rk2
import numpy as np

## Introduction to Electric Force and Electric Fields

In order to understand electric fields, first we need to define what a field is. According to the text, a field in phyics is "a phyiscal quantity whose value depends on (or is a function of) position, relative to the source of the field." An electric field has the units Newtons/Coulomb. With this in mind, and the definition of a field, you can start to make the connection that an electric field is used to figure out electric forces. The field allows you to calculate the force at a given test location, given a configuration of source charges, independent of the value of the charge at the test location. Recall Coulomb's Law:
\begin{equation*}
\vec{F} = Q q k{\Delta\vec{r} \over |\Delta\vec{r}|^3}
\end{equation*}
Where $Q$ is the test charge, $q$ is the source charge, $k = 9 \times 10^9$, and the relative position $\Delta\vec{r}$ is defined by the following:
\begin{equation*}
\Delta\vec{r} = \vec{r}_{fp} - \vec{r}_{s}
\end{equation*}
The keyword here is <i>relative</i>, as this position is the difference in the position vectors (aka coordinates), where $\vec{r}_{s}$ is the position of $q$ and $\vec{r}_{fp}$ is the position $Q$.
The result of this equation is a vector. Recall that the magnitude of a vector can be mathematically represented with the vertical bars i.e., $|\vec{v}| = \textbf{v}$

With all of this in mind, the above equation will give the vector force on test charge $Q$ due to source charge $q$ based on the charges' relative positions.


This can be easily modified for multiple source charges with the following:
\begin{equation*}
\vec{F}(\vec{r}) = k Q {\sum_{i=1}^n {q_i \over |\Delta\vec{r}_{i}|^3}} \Delta\vec{r}_{i}
\end{equation*}
This equation represents the vector sum of the forces due to $q_n$ source charges acting on a test charge $Q$ at location $\vec{r}$.

With this in mind, we can derive the equation for the net electric field $\vec{E}$ at point $\vec{r}_{fp}$ due to $q_n$ source charges by dividing the equation above by $Q$.
\begin{equation*}
\vec{E}(\vec{r}) = k {\sum_{i=1}^n {q_i \over |\Delta\vec{r}_{i}|^3}} \Delta\vec{r}_{i}
\end{equation*}

This is the equation we will be using for calculating the net electric field at a test point moving forward. You can find the electric force on any charge $Q$ at the location $\vec{r}$ by multiplying the result of $\vec{E}(\vec{r})$ by $Q$.

The code block below is an example of this using a basic electric dipole. The first value printed is the vector form of the electric field $\vec{E}(\vec{r})$, and the second number is the magnitude of this vector $|\vec{E}(\vec{r})|$.

Run the cells as they are written. Then, adjust the values for the charges and their positions (try two of the same sign with symmetry, try two with different magnitudes and signs, random locations in space, add more than just two charges, etc.). While experimenting with this, you can also adjust the test point location by changing the coordinates inside that vector.

In [21]:
test_point = vector(0,0,0)
q1 = Charge(charge=2e-1, position = vector(0,1,0))
q2 = Charge(charge=-2e-1, position = vector(0,-1,0))
charges = [q1, q2]

e_fields = [q.efield(test_point) for q in charges]

e_field_net = vector(0,0,0)

for i in e_fields:
    e_field_net += i
e
print(e_field_net)
mag(e_field_net)

<0, -3.6e+09, 0>


3600000000.0

The `canvas()` call below creates a VPython canvas object which is necessary whenever wanting to make visualizations. This creates an empty space which subsequent object calls will be added to.

In [22]:
canvas()

<IPython.core.display.Javascript object>

The two objects created below are 3D spheres which get plotted on our blank canvas above. Each sphere's name indicates the charge it represents. The spheres are passed `pos` which is the location of its center. They are also passed `radius` which is based on the magnitude of the charge. Finally, the colors are assigned with the convention that red is positive and blue is negative.

In [23]:
q1sphere = sphere(pos = q1.position, radius = q1.charge, color = color.red)
q2sphere = sphere(pos = q2.position, radius = q2.charge, color = color.blue)

Below we create an arrow representing the vector $\vec{E}(\vec{r})$ calculated above. The arrow is an object with similar attributes to the sphere. The multiplication in the `axis=` portion just scales the vector down so it's size is reasonable in relation to the size of the charges. Even scaling the vector down results in large vectors for points close to the positive charge. 

In [24]:
e_field_vector = arrow(pos=test_point, axis=e_field_net*10**-9, shaftwidth=.08)

We can update the test point in the cell above, rerun that cell, then rerun the cell above and add another arrow to the graph. This isn't the most efficient way to visualize the electric field. Because the electric field exists at any location in space, it would be impossible to draw enough of these arrows and eventually the arrows get cluttered. We introduce a new way of visualizing the electric field below.

## Drawing Electric Field Lines

Recall that our formula for the vector net electric field at test point/field point $\vec{r}_{fp}$ is:
\begin{equation*}
\vec{E}(\vec{r}) = k {\sum_{i=1}^n {q_i \over |\Delta\vec{r}_{i}|^3}} \Delta\vec{r}_{i}
\end{equation*}

Where each $\Delta \vec{r}_i$ equals $\vec{r}_{fp} - \vec{r}_i$ which is the relative position from source charge $q_i$ to the field point $\vec{r}_{fp}$.

Electric field lines are a good way to graphically represent electric fields. Lines start on positive charges and end on negative charges. The closer together lines are, the stronger the electric field is at those points. Electric field lines can be drawn by numerically solving the follow differential equation:
\begin{equation*}
{d\vec{r} \over ds} = {\vec{E(\vec{r}}) \over |\vec{E(\vec{r})}|}
\end{equation*}

Each of the $\vec{r}_n$ points generated by this equation are points along a curve that is defined by the above equation. We can use Euler's method of numerically estimating solutions to differential equations to solve the equation above with the following equation:
\begin{equation*}
\vec{r}_{n+1} = \vec{r}_n + {\vec{E(\vec{r_n}}) \over |\vec{E(\vec{r_n})}|}\Delta s
\end{equation*}

In the cell below we continue working with our dipole we created above. Recall that the charges are named `q1` and `q2`. Execute the cell below to print out the information about each charge to the console to ensure we are working with what we think we are. You can update object attributes by running lines like `q1.charge = 2e-2` etc.

In [25]:
print(q1.charge)
print(q1.position)
print(q2.charge)
print(q2.position)

0.2
<0, 1, 0>
-0.2
<0, -1, 0>


First we create an empty canvas.

In [26]:
canvas()

<IPython.core.display.Javascript object>

Then add spheres representing our charges.

In [27]:
q1sphere = sphere(pos = q1.position, radius = q1.charge, color = color.red)
q2sphere = sphere(pos = q2.position, radius = q2.charge, color = color.blue)

Add the charge objects to a list in order to compute the field due to the two charges with each starting point.

In [28]:
charges = [q1,q2]

The following cell generates the list of points that will be used to generate a field line starting from point $\vec{r}_0$ which is stored in the variable `r_0`. 

We calculate the first approximation outside the loop. Each approximation will be stored in the variable `r_n` and will be added to a list `lst` through the `.append()` method. 

This will store all the points which can then be used to create the field line. In order to prevent the `while` loop from never exiting, we provide stop conditions. One of them is based on the distance from the newest calculated point to the negative charge. This prevents the loop from hitting a divide by zero error when the field point is right on top of the charge. The other stop condition is related to the value of s which is accumulating the length of the line over each iteration. The cutoff for this length is 5, but the value can be adjusted depending on the location of the starting point of the field line. Each of the values calculated in the loop is appended to the list `lst`.

In [19]:
ds = .01
s = 0
min_dist = .2
r_0 = vector(0.173205080757,.9,0)
lst = []
lst.append(r_0)
r_n = r_0 + hat(q2.efield(r_0)+q1.efield(r_0))*ds
lst.append(r_n)
s+=ds
dist = mag(q2.position - r_n)
while ((dist > min_dist) and (s < 5)):
    r_n = euler(charges, r_n,ds)
    dist = mag(q2.position - r_n)
    s+=ds
    lst.append(r_n)
curve(lst)

In order to create a more complete picture, we must generate a list of starting points in different planes. The loop below does this using the vector rotate function in VPython.

In [10]:
r_0s = []
for n in range(0,7):
    r_0_x = q1.position + rotate(vector(0,-0.2,0), angle = n *np.pi/3, axis=vector(1,0,0))
    r_0_y = q1.position + rotate(vector(0.2,0,0), angle = n*np.pi/3, axis=vector(0,1,0))
    r_0_z = q1.position + rotate(vector(0,-0.2,0), angle = n*np.pi/3, axis=vector(0,0,1))
    r_0s.append(r_0_x)
    r_0s.append(r_0_y)
    r_0s.append(r_0_z)

In [ ]:
for r0 in r_0s:
    lst = []
    ds = .01
    s = 0
    min_dist = .2
    lst.append(r0)
    r_n = rk2(charges,r0,ds)
    dist = mag(q1.position - r_n)
    lst.append(r_n)
    while ((dist > min_dist) and (s < 10)):
        r_n = rk2(charges, r_n, ds)
        dist = mag(q1.position - r_n)
        s+=ds
        lst.append(r_n)
    curve(lst)